1. setup

Importera in bibliotek
Sätt k-värde 
Sät top_n-värde

In [445]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix
from sklearn.metrics import mean_absolute_error

Nedan lägger vi till csv-filerna med Användares betyg och spelen 
....

In [4]:
df_ratings = pd.read_csv("dataset/user_ratings.csv")
df_games = pd.read_csv("dataset/games.csv")
df_ratings.info()
#df_games.info()
df = pd.merge(df_ratings, df_games[['BGGId', 'Name']], on='BGGId', how='left')
df.info()   


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18942215 entries, 0 to 18942214
Data columns (total 3 columns):
 #   Column    Dtype  
---  ------    -----  
 0   BGGId     int64  
 1   Rating    float64
 2   Username  object 
dtypes: float64(1), int64(1), object(1)
memory usage: 433.6+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 18942215 entries, 0 to 18942214
Data columns (total 4 columns):
 #   Column    Dtype  
---  ------    -----  
 0   BGGId     int64  
 1   Rating    float64
 2   Username  object 
 3   Name      object 
dtypes: float64(1), int64(1), object(2)
memory usage: 722.6+ MB


In [5]:
df_ratings

,BGGId,Rating,Username
0,213788,8.0,Tonydorrf
1,213788,8.0,tachyon14k
2,213788,8.0,Ungotter
3,213788,8.0,brainlocki3
4,213788,8.0,PPMP
...,...,...,...
18942210,165521,3.0,rseater
18942211,165521,3.0,Bluefox86
18942212,165521,3.0,serginator
18942213,193488,1.0,CaptainCattan


In [6]:
df.describe()

,BGGId,Rating
count,1.894222e+07,1.894222e+07
mean,1.099533e+05,7.130957e+00
std,9.280217e+04,1.543023e+00
min,1.000000e+00,1.000000e-04
25%,1.611000e+04,6.000000e+00
50%,1.075290e+05,7.000000e+00
75%,1.813040e+05,8.000000e+00
max,3.491610e+05,1.000000e+01


2 Preprocessing

Handle missing valuee

In [7]:
any_null_values = df.isnull().any()
any_null_values

null_counts = df.isnull().sum()
print(null_counts)

BGGId        0
Rating       0
Username    63
Name         0
dtype: int64


In [8]:
df = df.dropna()

In [9]:
df.isnull().any()

BGGId       False
Rating      False
Username    False
Name        False
dtype: bool

In [10]:
df

,BGGId,Rating,Username,Name
0,213788,8.0,Tonydorrf,Ascension: Year Three Collector's Edition
1,213788,8.0,tachyon14k,Ascension: Year Three Collector's Edition
2,213788,8.0,Ungotter,Ascension: Year Three Collector's Edition
3,213788,8.0,brainlocki3,Ascension: Year Three Collector's Edition
4,213788,8.0,PPMP,Ascension: Year Three Collector's Edition
...,...,...,...,...
18942210,165521,3.0,rseater,Rory's Story Cubes: Medic
18942211,165521,3.0,Bluefox86,Rory's Story Cubes: Medic
18942212,165521,3.0,serginator,Rory's Story Cubes: Medic
18942213,193488,1.0,CaptainCattan,Airlines


In [11]:
duplicates = df.duplicated().value_counts()
duplicates

False    18942152
dtype: int64

In [12]:
duplicates.describe()

count           1.0
mean     18942152.0
std             NaN
min      18942152.0
25%      18942152.0
50%      18942152.0
75%      18942152.0
max      18942152.0
dtype: float64

Remove duplicate ratings for the same game by the same user (due to different version of the game etc)

In [13]:
# Calculate the mean rating for each group of duplicate rows
mean_ratings = df.groupby(['Username', 'BGGId'])['Rating'].mean()

In [14]:
mean_ratings

Username        BGGId 
 Fu_Koios       112092    9.0
                223033    9.0
 beastvol       13        8.0
                118       7.0
                278       7.0
                         ... 
Æleksandr Þræð  2399      7.0
                2932      6.0
                5451      8.0
                5554      7.0
                9962      8.0
Name: Rating, Length: 18909465, dtype: float64

In [15]:
# Filter the DataFrame for the specific user and item
specific_user_item_ratings = df[(df['Username'] == 'hariseldoncwb') & (df['BGGId'] == 135382)]

# Calculate the mean rating for the specific user and item
mean_rating_specific_user_item = specific_user_item_ratings['Rating'].mean()
mean_rating_specific_user_item

5.5

In [16]:
# Merge the mean ratings back to the original DataFrame based on 'Username' and 'Name'
df_removed_duplicates = df.merge(mean_ratings, on=['Username', 'BGGId'], suffixes=('', '_mean'))

In [17]:
df_removed_duplicates

,BGGId,Rating,Username,Name,Rating_mean
0,213788,8.0,Tonydorrf,Ascension: Year Three Collector's Edition,8.0
1,213788,8.0,tachyon14k,Ascension: Year Three Collector's Edition,8.0
2,213788,8.0,Ungotter,Ascension: Year Three Collector's Edition,8.0
3,213788,8.0,brainlocki3,Ascension: Year Three Collector's Edition,8.0
4,213788,8.0,PPMP,Ascension: Year Three Collector's Edition,8.0
...,...,...,...,...,...
18942147,165521,3.0,rseater,Rory's Story Cubes: Medic,3.0
18942148,165521,3.0,Bluefox86,Rory's Story Cubes: Medic,3.0
18942149,165521,3.0,serginator,Rory's Story Cubes: Medic,3.0
18942150,193488,1.0,CaptainCattan,Airlines,1.0


In [18]:
# Filter the DataFrame to keep only the original rows (non-duplicates) and the calculated mean ratings
df_unique = df_removed_duplicates[~df_removed_duplicates.duplicated(subset=['Username', 'BGGId'], keep=False)]

In [19]:
df_unique

,BGGId,Rating,Username,Name,Rating_mean
0,213788,8.0,Tonydorrf,Ascension: Year Three Collector's Edition,8.0
1,213788,8.0,tachyon14k,Ascension: Year Three Collector's Edition,8.0
2,213788,8.0,Ungotter,Ascension: Year Three Collector's Edition,8.0
3,213788,8.0,brainlocki3,Ascension: Year Three Collector's Edition,8.0
4,213788,8.0,PPMP,Ascension: Year Three Collector's Edition,8.0
...,...,...,...,...,...
18942147,165521,3.0,rseater,Rory's Story Cubes: Medic,3.0
18942148,165521,3.0,Bluefox86,Rory's Story Cubes: Medic,3.0
18942149,165521,3.0,serginator,Rory's Story Cubes: Medic,3.0
18942150,193488,1.0,CaptainCattan,Airlines,1.0


In [285]:
# Drop the original 'Rating' column and rename the mean rating column
df_dropped = df_unique.drop(columns=['Rating']).rename(columns={'Rating_mean': 'Rating'})

In [286]:
# Reindex the columns to move 'Rating' to the second position
column_order = ['BGGId', 'Rating', 'Username', 'Name']
df_dropped = df_dropped.reindex(columns=column_order)

In [287]:
df_dropped

,BGGId,Rating,Username,Name
0,213788,8.0,Tonydorrf,Ascension: Year Three Collector's Edition
1,213788,8.0,tachyon14k,Ascension: Year Three Collector's Edition
2,213788,8.0,Ungotter,Ascension: Year Three Collector's Edition
3,213788,8.0,brainlocki3,Ascension: Year Three Collector's Edition
4,213788,8.0,PPMP,Ascension: Year Three Collector's Edition
...,...,...,...,...
18942147,165521,3.0,rseater,Rory's Story Cubes: Medic
18942148,165521,3.0,Bluefox86,Rory's Story Cubes: Medic
18942149,165521,3.0,serginator,Rory's Story Cubes: Medic
18942150,193488,1.0,CaptainCattan,Airlines


In [289]:
# Step 1: Group by 'Username' and calculate the average rating for each user
user_avg_ratings = df_dropped.groupby('Username')['Rating'].mean().reset_index()


Filter dataset

In [290]:
# Step 2: Merge the average ratings back into the original DataFrame
df_adjusted_mean = pd.merge(df_dropped, user_avg_ratings, on='Username', suffixes=('', '_avg'))


In [291]:
df_adjusted_mean

,BGGId,Rating,Username,Name,Rating_avg
0,213788,8.0,Tonydorrf,Ascension: Year Three Collector's Edition,7.367589
1,206593,8.0,Tonydorrf,Stronghold: Undead (Second Edition) – Kickstar...,7.367589
2,267333,7.0,Tonydorrf,Goetia: Nine Kings of Solomon,7.367589
3,249824,6.0,Tonydorrf,Munchkin Harry Potter Deluxe,7.367589
4,301085,6.0,Tonydorrf,Rebis,7.367589
...,...,...,...,...,...
18878027,195742,5.0,codyk4545,The Godfather: A New Don,5.000000
18878028,215065,2.0,UnrealTravis,75 Gnom' Street,2.000000
18878029,127096,3.0,Elianto1989,Metal Adventures,3.000000
18878030,126471,2.0,DaLanimilbus,Nowhere to Go,2.000000


In [292]:

# Step 3: Subtract the average rating from each rating to get the adjusted rating
df_adjusted_mean['Adjusted_Rating'] = df_adjusted_mean['Rating'] - df_adjusted_mean['Rating_avg']

column_order = ['BGGId', 'Rating', 'Rating_avg', 'Adjusted_Rating', 'Username', 'Name']
df_adjusted_mean = df_adjusted_mean.reindex(columns=column_order)
# Now df contains the adjusted ratings in the 'Adjusted_Rating' column

In [293]:
df_adjusted_mean

,BGGId,Rating,Rating_avg,Adjusted_Rating,Username,Name
0,213788,8.0,7.367589,0.632411,Tonydorrf,Ascension: Year Three Collector's Edition
1,206593,8.0,7.367589,0.632411,Tonydorrf,Stronghold: Undead (Second Edition) – Kickstar...
2,267333,7.0,7.367589,-0.367589,Tonydorrf,Goetia: Nine Kings of Solomon
3,249824,6.0,7.367589,-1.367589,Tonydorrf,Munchkin Harry Potter Deluxe
4,301085,6.0,7.367589,-1.367589,Tonydorrf,Rebis
...,...,...,...,...,...,...
18878027,195742,5.0,5.000000,0.000000,codyk4545,The Godfather: A New Don
18878028,215065,2.0,2.000000,0.000000,UnrealTravis,75 Gnom' Street
18878029,127096,3.0,3.000000,0.000000,Elianto1989,Metal Adventures
18878030,126471,2.0,2.000000,0.000000,DaLanimilbus,Nowhere to Go


In [294]:
# Group by 'Username' and count the number of rows for each group
grouped_user_counts = df_adjusted_mean.groupby('Username').size()
grouped_user_counts

Username
 Fu_Koios              2
 beastvol              9
 mycroft              14
 woh                   5
(mostly) harmless      1
                    ... 
zzzuzu                39
zzzvone               21
zzzxxxyyy             32
zzzzzane             154
Æleksandr Þræð        12
Length: 411312, dtype: int64

In [295]:
# Group by 'BGGId' and count the number of rows for each group
grouped_counts = df_adjusted_mean.groupby('BGGId').size()
grouped_counts

BGGId
1          5295
2           560
3         15077
4           342
5         18152
          ...  
347146       42
347521       71
348955       67
349131       66
349161        7
Length: 21925, dtype: int64

In [297]:
game_ratings = 3000

rating_counter = df_adjusted_mean['BGGId'].value_counts()

filtered_out = rating_counter[rating_counter < game_ratings].index

games_mask = df_adjusted_mean['BGGId'].isin(filtered_out)
#games_mask = df['BGGId'].isin(rating_counter[rating_counter < game_ratings].index)
#games_mask
df_adjusted_mean.drop(index=df_adjusted_mean[games_mask].index, inplace=True)

df_adjusted_mean

,BGGId,Rating,Rating_avg,Adjusted_Rating,Username,Name
44,36218,7.0,7.367589,-0.367589,Tonydorrf,Dominion
45,133473,6.0,7.367589,-1.367589,Tonydorrf,Sushi Go!
46,167791,9.0,7.367589,1.632411,Tonydorrf,Terraforming Mars
47,230802,8.0,7.367589,0.632411,Tonydorrf,Azul
48,174430,9.0,7.367589,1.632411,Tonydorrf,Gloomhaven
...,...,...,...,...,...,...
18852015,125548,10.0,10.000000,0.000000,brocespringsteen,Pixel Tactics
18852016,125548,10.0,10.000000,0.000000,Czerstwy,Pixel Tactics
18852018,125548,10.0,10.000000,0.000000,Shining Creations,Pixel Tactics
18852019,125548,10.0,10.000000,0.000000,MissTaluva,Pixel Tactics


In [298]:
user_ratings = 100

rating_counter = df_adjusted_mean['Username'].value_counts()

filtered_out = rating_counter[rating_counter < user_ratings].index

user_mask = df_adjusted_mean['Username'].isin(filtered_out)

df_adjusted_mean.drop(index=df_adjusted_mean[user_mask].index, inplace=True)

df_adjusted_mean

,BGGId,Rating,Rating_avg,Adjusted_Rating,Username,Name
386,178900,7.0,7.641304,-0.641304,PPMP,Codenames
387,14996,7.0,7.641304,-0.641304,PPMP,Ticket to Ride: Europe
388,40692,7.0,7.641304,-0.641304,PPMP,Small World
389,822,8.0,7.641304,0.358696,PPMP,Carcassonne
390,70323,7.0,7.641304,-0.641304,PPMP,King of Tokyo
...,...,...,...,...,...,...
18691910,634,10.0,10.000000,0.000000,AbsterPro,River Dragons
18691911,174973,10.0,10.000000,0.000000,AbsterPro,Boss Monster 2: The Next Level
18691912,191894,10.0,10.000000,0.000000,AbsterPro,Imagine
18691913,19526,10.0,10.000000,0.000000,AbsterPro,Elasund: The First City


In [299]:
# Group by 'Username' and count the number of rows for each group
grouped_user_counts = df_adjusted_mean.groupby('Username').size()
grouped_user_counts

Username
-=Yod@=-     201
-Johnny-     245
-mIDE-       356
-toni-       137
...Hammer    123
            ... 
zzyzxuk      118
zzzabiss     296
zzzhsm       171
zzzoren      141
zzzzzane     136
Length: 31482, dtype: int64

In [300]:
# Group by 'BGGId' and count the number of rows for each group
grouped_counts = df_adjusted_mean.groupby('BGGId').size()
grouped_counts

BGGId
1         3647
3         8711
5         9065
7         1751
10        5111
          ... 
316554    5818
317985    3028
318977    2779
328871    1826
329465    1812
Length: 1167, dtype: int64

In [301]:
grouped_by_username = df_adjusted_mean.groupby('Username').size().sort_values(ascending=False)
grouped_by_username

Username
leffe dubbel            1081
Hessu68                 1020
oldgoat3769967          1002
TomVasel                 996
Pandorzecza              953
                        ... 
RoN_Rancor               100
generationX              100
Nathan G                 100
starvinmartian_games     100
chsbellboy               100
Length: 31482, dtype: int64

In [302]:
df_adjusted_mean

,BGGId,Rating,Rating_avg,Adjusted_Rating,Username,Name
386,178900,7.0,7.641304,-0.641304,PPMP,Codenames
387,14996,7.0,7.641304,-0.641304,PPMP,Ticket to Ride: Europe
388,40692,7.0,7.641304,-0.641304,PPMP,Small World
389,822,8.0,7.641304,0.358696,PPMP,Carcassonne
390,70323,7.0,7.641304,-0.641304,PPMP,King of Tokyo
...,...,...,...,...,...,...
18691910,634,10.0,10.000000,0.000000,AbsterPro,River Dragons
18691911,174973,10.0,10.000000,0.000000,AbsterPro,Boss Monster 2: The Next Level
18691912,191894,10.0,10.000000,0.000000,AbsterPro,Imagine
18691913,19526,10.0,10.000000,0.000000,AbsterPro,Elasund: The First City


In [526]:
average_ratings = dict(zip(df_adjusted_mean['Username'], df_adjusted_mean['Rating_avg']))

Train test split

In [303]:
# Define a function to split the data for each user
def user_split(data, test_size=0.2):    
    train_to_concat = []
    test_to_concat = []
    for user, group in data.groupby('Username'):
        train_user, test_user = train_test_split(group, test_size=test_size, train_size=1-test_size)
        train_to_concat.append(train_user)
        test_to_concat.append(test_user)
    
    train = pd.concat(train_to_concat, ignore_index=True)
    test = pd.concat(test_to_concat, ignore_index=True)

    return train, test


In [304]:
# Split the data
train_set, test_set = user_split(df_adjusted_mean, test_size=0.2)

In [305]:
train_set

,BGGId,Rating,Rating_avg,Adjusted_Rating,Username,Name
0,157354,8.0,7.287829,0.712171,-=Yod@=-,Five Tribes
1,215841,8.0,7.287829,0.712171,-=Yod@=-,Exit: The Game – The Forgotten Island
2,46213,8.0,7.287829,0.712171,-=Yod@=-,Telestrations
3,131325,7.0,7.287829,-0.287829,-=Yod@=-,Timeline: General Interest
4,1829,8.0,7.287829,0.712171,-=Yod@=-,Risk 2210 A.D.
...,...,...,...,...,...,...
4754392,72321,6.8,7.393506,-0.593506,zzzzzane,The Networks
4754393,128882,6.0,7.393506,-1.393506,zzzzzane,The Resistance: Avalon
4754394,28143,7.0,7.393506,-0.393506,zzzzzane,Race for the Galaxy
4754395,1927,5.0,7.393506,-2.393506,zzzzzane,Munchkin


In [306]:
test_set

,BGGId,Rating,Rating_avg,Adjusted_Rating,Username,Name
0,203417,6.0,7.287829,-1.287829,-=Yod@=-,Exit: The Game – The Secret Lab
1,90419,7.5,7.287829,0.212171,-=Yod@=-,Airlines Europe
2,124361,8.0,7.287829,0.712171,-=Yod@=-,Concordia
3,7,7.5,7.287829,0.212171,-=Yod@=-,Cathedral
4,27173,7.0,7.287829,-0.287829,-=Yod@=-,Vikings
...,...,...,...,...,...,...
1204355,90419,10.0,7.393506,2.606494,zzzzzane,Airlines Europe
1204356,15062,8.0,7.393506,0.606494,zzzzzane,Shadows over Camelot
1204357,157354,10.0,7.393506,2.606494,zzzzzane,Five Tribes
1204358,12,9.0,7.393506,1.606494,zzzzzane,Ra


In [540]:
#Used to calculate MAE
actual_ratings_test_set = {}
for _, row in test_set.iterrows():
    username = row['Username']
    bggid = row['BGGId']
    rating = row['Rating']
    
    if username not in actual_ratings_test_set:
        actual_ratings_test_set[username] = {}
    
    actual_ratings_test_set[username][bggid] = rating

In [558]:
actual_ratings_train_set = {}
for _, row in train_set.iterrows():
    username = row['Username']
    bggid = row['BGGId']
    rating = row['Rating']
    
    if username not in actual_ratings_train_set:
        actual_ratings_train_set[username] = {}
    
    actual_ratings_train_set[username][bggid] = rating

In [548]:
#Used to calculate MAE
rated_items_test_set = {}
for index, row in test_set.iterrows():
    username = row['Username']
    bggid = row['BGGId']
    
    if username not in rated_items_test_set:
        rated_items_test_set[username] = []
    
    rated_items_test_set[username].append(bggid)

In [549]:
rated_items_train_set = {}
for index, row in train_set.iterrows():
    username = row['Username']
    bggid = row['BGGId']
    
    if username not in rated_items_train_set:
        rated_items_train_set[username] = []
    
    rated_items_train_set[username].append(bggid)

filtrera dataframe 

3 Similarity Calculation

 Skapa en matris av item-för-item-similarity baserat på cosine similarity

In [93]:
# Pivot the DataFrame so a user matrix, so rows represent users and columns represent items
user_item_matrix = train_set.pivot(index='Username', columns='BGGId', values='Rating').fillna(0)



In [94]:
user_item_matrix

BGGId,1,3,5,7,10,11,12,13,15,18,...,304420,306735,308765,312484,314040,316554,317985,318977,328871,329465
Username,,,,,,,,,,,,,,,,,,,,,
-=Yod@=-,0.0,0.0,0.0,7.5,0.0,0.0,0.0,7.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0
-Johnny-,0.0,0.0,7.0,0.0,4.0,6.0,0.0,7.0,0.0,6.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-mIDE-,0.0,8.0,0.0,0.0,0.0,7.5,8.0,7.0,0.0,8.0,...,0.0,0.0,0.0,9.0,0.0,0.0,0.0,8.0,0.0,0.0
-toni-,0.0,9.0,0.0,0.0,0.0,7.0,0.0,4.0,0.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...Hammer,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zzyzxuk,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0
zzzabiss,0.0,0.0,0.0,0.0,0.0,5.0,0.0,7.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zzzhsm,5.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [307]:
# Pivot the DataFrame so a user matrix, so rows represent users and columns represent items
user_item_matrix_with_nan = train_set.pivot(index='Username', columns='BGGId', values='Adjusted_Rating')

In [308]:
user_item_matrix_with_nan

BGGId,1,3,5,7,10,11,12,13,15,18,...,304420,306735,308765,312484,314040,316554,317985,318977,328871,329465
Username,,,,,,,,,,,,,,,,,,,,,
-=Yod@=-,NaN,NaN,NaN,NaN,NaN,-0.287829,NaN,-0.287829,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-Johnny-,-0.322245,-1.322245,1.677755,NaN,-1.322245,NaN,-0.322245,1.677755,2.677755,0.677755,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-mIDE-,NaN,0.964286,NaN,NaN,NaN,0.464286,0.964286,-0.035714,NaN,0.964286,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.964286,NaN,NaN
-toni-,NaN,2.552846,NaN,NaN,NaN,0.552846,NaN,-2.447154,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...Hammer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zzyzxuk,NaN,NaN,0.251701,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.251701,NaN,NaN,NaN,NaN,NaN,NaN
zzzabiss,NaN,NaN,NaN,NaN,NaN,-0.828194,NaN,1.171806,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
zzzhsm,-1.461988,NaN,1.538012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [567]:
# Used in recommend items

not_rated_items_train_set = {}
# Iterate over the rows of the DataFrame
for username, row in user_item_matrix_with_nan.iterrows():
    nan_bggid_list = []
    # Iterate over the columns (BGGIds)
    for bggid, value in row.items():
        # Check if the value is NaN
        if pd.isna(value):
            nan_bggid_list.append(bggid)
    # Store the list of BGGId values with NaN for the current Username in the dictionary
    not_rated_items_train_set[username] = nan_bggid_list


In [95]:
# Transpose the user-item matrix to obtain an item-user matrix
item_user_matrix = user_item_matrix.T

In [96]:
item_user_matrix

Username,-=Yod@=-,-Johnny-,-mIDE-,-toni-,...Hammer,.JcK.,0010,0b1_Ita,0ddjob,0hoHj,...,zzas,zzay,zzeroparticle,zztap,zzyzewitsch,zzyzxuk,zzzabiss,zzzhsm,zzzoren,zzzzzane
BGGId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,7.0,0.0,0.0,5.0,0.0,0.0
3,0.0,0.0,8.0,9.0,0.0,6.0,0.0,7.0,0.0,7.9,...,7.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,6.0,5.0,7.0,0.0,8.0,0.0,0.0
7,7.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316554,0.0,0.0,0.0,0.0,7.0,0.0,9.0,0.0,0.0,10.0,...,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0
317985,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
318977,7.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [309]:
item_user_matrix_adjusted = user_item_matrix_with_nan.T

In [310]:
item_user_matrix_adjusted

Username,-=Yod@=-,-Johnny-,-mIDE-,-toni-,...Hammer,.JcK.,0010,0b1_Ita,0ddjob,0hoHj,...,zzas,zzay,zzeroparticle,zztap,zzyzewitsch,zzyzxuk,zzzabiss,zzzhsm,zzzoren,zzzzzane
BGGId,,,,,,,,,,,,,,,,,,,,,
1,NaN,-0.322245,NaN,NaN,NaN,2.103896,NaN,NaN,NaN,2.17541,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.461988,NaN,NaN
3,NaN,-1.322245,0.964286,2.552846,NaN,-0.896104,NaN,-0.097356,NaN,0.07541,...,0.453333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,1.677755,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.253333,NaN,NaN,-0.832021,-1.37859,0.251701,NaN,1.538012,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,-1.322245,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316554,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.17541,...,NaN,NaN,NaN,2.167979,NaN,NaN,NaN,NaN,NaN,NaN
317985,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.17541,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
318977,NaN,NaN,0.964286,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [354]:
def calculate_adjusted_cosine_similarity(matrix_transposed):
    sparse_matrix = csr_matrix(matrix_transposed.values)
    num_items = sparse_matrix.shape[0]

    # Initialize an empty similarity matrix
    similarity_matrix = np.zeros((num_items, num_items))

    # Iterate over each row (item) in the transposed matrix by index
    for i in range(num_items):
        for j in range(i, num_items):  # Only compute upper triangle (similarity_matrix is symmetric)
            # Get rows by numeric position using csr_matrix row slicing
            row_i = sparse_matrix.getrow(i)
            row_j = sparse_matrix.getrow(j)

            # Find common indices and adjust to exclude columns with NaN values
            common_indices = np.where(~np.isnan(row_i.toarray()) & ~np.isnan(row_j.toarray()))[1]
    
            if common_indices.size > 0:
                # Get non-missing values for common users
                non_missing_values_i = row_i[:, common_indices].toarray().flatten()
                non_missing_values_j = row_j[:, common_indices].toarray().flatten()

                # Check if both arrays have non-zero length
                if len(non_missing_values_i) > 0 and len(non_missing_values_j) > 0:
                    similarity = cosine_similarity([non_missing_values_i], [non_missing_values_j])[0, 0]
                    similarity_matrix[i, j] = similarity
                    similarity_matrix[j, i] = similarity

    return similarity_matrix

In [364]:
item_similarity = calculate_adjusted_cosine_similarity(item_user_matrix_adjusted)

In [369]:
# Create a DataFrame with item names as both rows and columns
item_similarity_df = pd.DataFrame(item_similarity, index=item_user_matrix_adjusted.index, columns=item_user_matrix_adjusted.index)

# Now item_item_similarity_df is your item-item similarity matrix with item names as both row and column labels
# Set similarity to 0 for each item to itself
np.fill_diagonal(item_similarity_df.values, -5)
item_similarity_df

BGGId,1,3,5,7,10,11,12,13,15,18,...,304420,306735,308765,312484,314040,316554,317985,318977,328871,329465
BGGId,,,,,,,,,,,,,,,,,,,,,
1,-5.000000,0.340656,0.390060,-0.061612,0.055123,0.122371,0.397868,0.057064,0.033246,0.017704,...,0.348507,0.487772,0.437120,0.281959,0.442883,0.372876,0.376988,0.362176,0.150299,0.061127
3,0.340656,-5.000000,0.343598,-0.079699,0.029441,0.166632,0.481848,0.097839,-0.046591,0.036892,...,0.392138,0.421592,0.389602,0.362043,0.327983,0.398687,0.455530,0.362501,0.400704,0.035695
5,0.390060,0.343598,-5.000000,-0.107712,0.105199,0.214517,0.360867,0.188784,0.105219,0.133812,...,0.160726,0.232109,0.177217,0.226871,0.318987,0.285098,0.330810,0.342941,0.373151,0.174379
7,-0.061612,-0.079699,-0.107712,-5.000000,0.075219,-0.027759,-0.084996,-0.027619,-0.067189,-0.061406,...,-0.306879,-0.260714,-0.358628,-0.404293,-0.468613,-0.345082,-0.376806,-0.177436,-0.237257,0.158268
10,0.055123,0.029441,0.105199,0.075219,-5.000000,0.178086,0.009451,0.141451,0.112599,0.094001,...,-0.299844,-0.313365,-0.329686,-0.236630,-0.212215,-0.235163,-0.229372,-0.136044,-0.153873,0.041880
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316554,0.372876,0.398687,0.285098,-0.345082,-0.235163,-0.097468,0.385982,-0.239466,-0.014926,-0.080419,...,0.639164,0.688401,0.730106,0.775066,0.783849,-5.000000,0.780077,0.652878,0.755105,0.199125
317985,0.376988,0.455530,0.330810,-0.376806,-0.229372,0.014929,0.431348,-0.266904,-0.182984,-0.096529,...,0.619399,0.682106,0.737772,0.763602,0.788045,0.780077,-5.000000,0.732845,0.741310,0.131939
318977,0.362176,0.362501,0.342941,-0.177436,-0.136044,0.030643,0.411722,-0.151539,-0.021729,0.093569,...,0.531504,0.633998,0.630305,0.672121,0.693972,0.652878,0.732845,-5.000000,0.686111,0.119940


In [370]:
# Convert similarity to distances (inversely proportional to similarities)
distance_matrix = 1 - item_similarity_df

In [371]:
distance_matrix

BGGId,1,3,5,7,10,11,12,13,15,18,...,304420,306735,308765,312484,314040,316554,317985,318977,328871,329465
BGGId,,,,,,,,,,,,,,,,,,,,,
1,6.000000,0.659344,0.609940,1.061612,0.944877,0.877629,0.602132,0.942936,0.966754,0.982296,...,0.651493,0.512228,0.562880,0.718041,0.557117,0.627124,0.623012,0.637824,0.849701,0.938873
3,0.659344,6.000000,0.656402,1.079699,0.970559,0.833368,0.518152,0.902161,1.046591,0.963108,...,0.607862,0.578408,0.610398,0.637957,0.672017,0.601313,0.544470,0.637499,0.599296,0.964305
5,0.609940,0.656402,6.000000,1.107712,0.894801,0.785483,0.639133,0.811216,0.894781,0.866188,...,0.839274,0.767891,0.822783,0.773129,0.681013,0.714902,0.669190,0.657059,0.626849,0.825621
7,1.061612,1.079699,1.107712,6.000000,0.924781,1.027759,1.084996,1.027619,1.067189,1.061406,...,1.306879,1.260714,1.358628,1.404293,1.468613,1.345082,1.376806,1.177436,1.237257,0.841732
10,0.944877,0.970559,0.894801,0.924781,6.000000,0.821914,0.990549,0.858549,0.887401,0.905999,...,1.299844,1.313365,1.329686,1.236630,1.212215,1.235163,1.229372,1.136044,1.153873,0.958120
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316554,0.627124,0.601313,0.714902,1.345082,1.235163,1.097468,0.614018,1.239466,1.014926,1.080419,...,0.360836,0.311599,0.269894,0.224934,0.216151,6.000000,0.219923,0.347122,0.244895,0.800875
317985,0.623012,0.544470,0.669190,1.376806,1.229372,0.985071,0.568652,1.266904,1.182984,1.096529,...,0.380601,0.317894,0.262228,0.236398,0.211955,0.219923,6.000000,0.267155,0.258690,0.868061
318977,0.637824,0.637499,0.657059,1.177436,1.136044,0.969357,0.588278,1.151539,1.021729,0.906431,...,0.468496,0.366002,0.369695,0.327879,0.306028,0.347122,0.267155,6.000000,0.313889,0.880060


Calculate nearest neighbor

In [372]:
def find_nearest_neighbors(distance_matrix, k, BGGId):

    # Create a NearestNeighbors object
    nearest_neighbors_model = NearestNeighbors(n_neighbors=k, metric='precomputed')

    # Fit the model using the item-item similarity matrix
    nearest_neighbors_model.fit(distance_matrix.values)

    # Find the index of the item
    item_index = distance_matrix.index.get_loc(BGGId)

    # Find the distance and indices of the nearest neighbors
    distances, neighbor_indices = nearest_neighbors_model.kneighbors([distance_matrix.iloc[item_index]], return_distance=True)

    return distances, neighbor_indices

In [469]:
def get_rated_items_for_user(username):
    # Filter the matrix for the specified user and non-zero ratings
    user_ratings = item_user_matrix.loc[:, username]
    non_zero_ratings = user_ratings[user_ratings != 0]
    
    return non_zero_ratings

In [582]:
test_items = get_rated_items_for_user("-=Yod@=-")
test_items.index

Int64Index([     7,     13,     42,     50,     74,     93,    172,    432,
               463,    466,
            ...
            256226, 257499, 257501, 263918, 266192, 271896, 284083, 285774,
            291457, 318977],
           dtype='int64', name='BGGId', length=160)

In [585]:
test_items2 = rated_items_train_set["-=Yod@=-"]
test_items2.sort()
test_items2

[11,
 13,
 42,
 47,
 93,
 172,
 217,
 320,
 432,
 466,
 478,
 503,
 715,
 822,
 1502,
 1829,
 1917,
 2381,
 2511,
 2651,
 2653,
 3076,
 3201,
 3955,
 5782,
 8098,
 9209,
 9220,
 9625,
 9674,
 10093,
 10547,
 12333,
 12493,
 12692,
 14105,
 14996,
 15062,
 15512,
 15818,
 15987,
 17223,
 17226,
 18602,
 18803,
 18833,
 20551,
 20963,
 22038,
 22141,
 24480,
 25613,
 25821,
 27162,
 28143,
 31260,
 31481,
 35677,
 36218,
 37111,
 37380,
 37904,
 38453,
 39463,
 39856,
 40692,
 40834,
 41114,
 42452,
 43443,
 45315,
 46213,
 48726,
 54043,
 54625,
 54998,
 65244,
 66188,
 68448,
 72287,
 83330,
 84419,
 84876,
 90137,
 91536,
 92828,
 97207,
 100423,
 100901,
 102548,
 103343,
 103885,
 103886,
 104363,
 118048,
 120677,
 121408,
 121410,
 124742,
 126163,
 128621,
 128667,
 128882,
 129622,
 131325,
 131357,
 131835,
 139030,
 143884,
 145189,
 146021,
 146508,
 146652,
 146791,
 147020,
 148951,
 150376,
 154203,
 154458,
 155426,
 156858,
 157354,
 161970,
 163412,
 163967,
 167355,
 1

In [561]:
def find_rated_neighbors(neighbor_distances, neighbor_indices, rated_items):
    neighbor_indices = neighbor_indices[0]
    neighbor_distances = neighbor_distances[0]
    neighbors = item_user_matrix.iloc[neighbor_indices].index.tolist()
    rated_neighbors = []
    rated_neighbors_distances = []

    for i in range(len(neighbors)):
        if neighbors[i] in rated_items.index:
        #if neighbors[i] in rated_items:
            rated_neighbors.append(neighbors[i])
            rated_neighbors_distances.append(neighbor_distances[i])
    
    return rated_neighbors_distances, rated_neighbors

In [562]:
def get_predicted_rating(item, username, k, distance_matrix):
    rated_items = get_rated_items_for_user(username)
    #rated_items = rated_items_train_set[username]
    distances, neighbor_indices = find_nearest_neighbors(distance_matrix, k, item)
    rated_neighbors_distances, rated_neighbors = find_rated_neighbors(distances, neighbor_indices, rated_items)

    if len(rated_neighbors) != 0:
        numerator = 0
        denominator = 0
        for i in range(len(rated_neighbors)):
            neighbor = rated_neighbors[i]
            weight = 1 - rated_neighbors_distances[i] #transform distance into similarity
            user_rating = rated_items.loc[neighbor]
            #user_rating = actual_ratings_train_set[username][neighbor]
            numerator += (weight * user_rating)
            denominator += weight
        predicted_rating_item = numerator / denominator
        return predicted_rating_item
    else:
        average_rating = average_ratings[username]
        return average_rating

In [570]:
def recommend_items(distance_matrix, username, k, top_N):
    predicted_ratings = []
    predicted_items = []
    #not_rated_items = get_not_rated_items_for_user(username)
    not_rated_items = not_rated_items_train_set[username]

    for item in not_rated_items:
    #for item in not_rated_items.index:
        predicted_rating_item = get_predicted_rating(item, username, k, distance_matrix)
        predicted_ratings.append(predicted_rating_item)
        predicted_items.append(item)  
    
    recommended_items = pd.DataFrame(predicted_ratings, index=predicted_items, columns=['Predicted rating'])
    recommended_items.index.name = 'BGGId'
    sorted_recommendations = recommended_items.sort_values(by=['Predicted rating'], ascending=False)
    top_N_recommendations = sorted_recommendations.head(top_N)
    top_N_list = top_N_recommendations.index.tolist()

    return top_N_list, top_N_recommendations

In [261]:
def calculate_precision(recommended_items, relevant_items):
    true_positives = 0
    for item in recommended_items:
        if item in relevant_items:
            true_positives += 1
            print(item)
    
    print("antal true positives: ", true_positives)
    return true_positives / len(recommended_items)


In [553]:
#Dont need anymore, saved in dictionary instead

# def get_items_in_test_set_for_user(username):
#     items_df = test_set.loc[test_set['Username'] == username]
#     return items_df['BGGId'].tolist()

In [551]:
def calculate_mean_absolute_error(username, k, distance_matrix):
    actual_ratings = []
    predicted_ratings = []
    #rated_items_testset = get_items_in_test_set_for_user(username)
    rated_items_testset = rated_items_test_set[username]

    for item in rated_items_testset:
        predicted_ratings.append(get_predicted_rating(item, username, k, distance_matrix))
        actual_rating = actual_ratings_test_set[username][item]
        actual_ratings.append(actual_rating)
    
    mae = mean_absolute_error(actual_ratings, predicted_ratings)
    return mae

In [572]:
def get_user_sample(random_seed, size):
    sample = set()
    num_users, num_items = user_item_matrix.shape
    np.random.seed(random_seed)
    while len(sample) < 1000:
        random_user_index = np.random.randint(0, num_users)
        random_user = user_item_matrix.index[random_user_index]
        sample.add(random_user)
    return sample
    

In [575]:
user_sample = get_user_sample(0, 1000)
user_sample

{'Davord',
 'proz1615',
 'rainbowflight',
 'TheMadWelshWizard',
 'Kingbubba',
 'victorwfreire',
 'White Elk',
 'Patrice_jps',
 'namka',
 'danielbonito',
 'torsten354',
 'Dzastus',
 'ilopez10',
 'Bracca',
 'Tiril',
 'MindAstray',
 'Grantelith',
 'bkindt',
 'AlasDemigod',
 'randall',
 'calbond',
 'Phoenix1309',
 'Malakizedek',
 'GROM TROM',
 'Craigus69',
 'legendno6',
 'dbreen27',
 'buffy2295',
 'Xadocs',
 'Volstag',
 'Fuchs78',
 'DiggityDog',
 'Jehofer',
 'Rudens',
 'Melosh007',
 'Wiles37',
 'kenshinku',
 'jetimmerman',
 'rotosherra',
 'etotheipiplusone',
 'samc1078',
 'kreten',
 'shawnspring',
 'sojapeti',
 'uprightgorilla',
 'Indy418',
 'GamesNightNews',
 'RobbyKoreman',
 'Pantherrob96',
 'SuperAbobo',
 'make4U',
 'elbanshee',
 'mauserem',
 'PingisPajas',
 'pastorarod',
 'ulycious',
 'tiwaishi',
 'awanano',
 'midge11',
 'Luis Francisco',
 'Burrito',
 'MerelyRachel',
 'Morvel',
 'EdgarMolas',
 'messire76',
 'mkemp217',
 'puppeteer',
 'joflynn',
 'thorbman',
 'Mihealsick',
 'doktorscien

In [577]:
def run_mean_absolute_error_calculation(k, users):
    results = []
    num_users, num_items = user_item_matrix.shape
    for user in users:
        mae = calculate_mean_absolute_error(user, k, distance_matrix)
        results.append(mae)
    print("Mean absolute error with k = ", k ,"and num_users =", len(users),":", mae)
    return np.sum(results) / len(users)


In [579]:
# mae_k5 = run_mean_absolute_error_calculation(5, user_sample)
# mae_k10 = run_mean_absolute_error_calculation(10, user_sample)

Mean absolute error with k =  5 and num_users = 1000 : 1.214773270993859
Mean absolute error with k =  10 and num_users = 1000 : 0.9687931172532076


In [550]:
test_mae = calculate_mean_absolute_error("-=Yod@=-", 20, distance_matrix)
test_mae

0.82575377384489

In [571]:
test_recommended_items, test_rec_df = recommend_items(distance_matrix, "-=Yod@=-", 20, 30)
test_recommended_items

[185343,
 283948,
 244711,
 233398,
 124361,
 246684,
 246900,
 229853,
 253344,
 256960,
 265188,
 266810,
 276025,
 203993,
 163068,
 2346,
 25021,
 193738,
 312484,
 308765,
 192135,
 296151,
 177736,
 176734,
 161614,
 248562,
 215341,
 233078,
 316554,
 237182]

In [408]:
relevant_items = test_set.loc[test_set['Rating'] >= 6]
#relevant_items = test_set.loc[test_set['Adjusted_Rating'] > 0]

In [409]:
relevant_items

,BGGId,Rating,Rating_avg,Adjusted_Rating,Username,Name
0,203417,6.0,7.287829,-1.287829,-=Yod@=-,Exit: The Game – The Secret Lab
1,90419,7.5,7.287829,0.212171,-=Yod@=-,Airlines Europe
2,124361,8.0,7.287829,0.712171,-=Yod@=-,Concordia
3,7,7.5,7.287829,0.212171,-=Yod@=-,Cathedral
4,27173,7.0,7.287829,-0.287829,-=Yod@=-,Vikings
...,...,...,...,...,...,...
1204355,90419,10.0,7.393506,2.606494,zzzzzane,Airlines Europe
1204356,15062,8.0,7.393506,0.606494,zzzzzane,Shadows over Camelot
1204357,157354,10.0,7.393506,2.606494,zzzzzane,Five Tribes
1204358,12,9.0,7.393506,1.606494,zzzzzane,Ra


In [248]:
def get_relevant_items_for_user(username):
    relevant_items_df = relevant_items.loc[relevant_items['Username'] == username]
    return relevant_items_df['BGGId'].tolist()

In [475]:
# Username
# leffe dubbel            1081
# Hessu68                 1020
# oldgoat3769967          1002
# TomVasel                 996
# Pandorzecza              953
#                         ... 
# RoN_Rancor               100
# generationX              100
# Nathan G                 100
# starvinmartian_games     100
# chsbellboy               100

#users: -Johnny-	-mIDE-	-toni-	...Hammer	
test_recommended_items_2, test_recommended_items_2_df = recommend_items(distance_matrix, "leffe dubbel", 50, 10)
test_relevant_items_2 = get_relevant_items_for_user("leffe dubbel")
test_precision_2 = calculate_precision(test_recommended_items_2, test_relevant_items_2)
test_precision_2

39351
875
175914
2346
175640
161533
93
62219
192836
193738
antal true positives:  10


1.0

In [414]:
len(test_relevant_items_2)

152

In [476]:
test_recommended_items_2_df

,Predicted rating
BGGId,
39351,8.119540
875,8.076252
175914,7.950966
2346,7.909610
175640,7.892101
161533,7.869614
93,7.839399
62219,7.836585
192836,7.827520


Nedan skriver vi funktion som visar resultat av rekomendation i form av en sorts lista baserat på recommender funktionen

Nedan här skriver vi evalueringsfunktioner av modellen och utskrift 